In [5]:
import talib as ta
import pandas as pd
import numpy as np

In [6]:
# read the data from the CSV files
train_data = pd.read_csv('train_data_50.csv') 

# change date into datetime objects
train_data['Date'] = pd.to_datetime(train_data['Date'])

# set indexes 
train_data.set_index(["Ticker", "Date"], inplace=True)

# sort data by tickers
tickers = sorted(train_data.index.get_level_values('Ticker').unique())

In [7]:
#create array of close prices 
tickers = sorted(train_data.index.get_level_values('Ticker').unique())

open_prices = []

for ticker in tickers:
    stock_close_data = train_data.loc[ticker]["Open"]
    open_prices.append(stock_close_data.values)

open_prices = np.stack(open_prices)
print(open_prices.shape)
print(open_prices)

# array for final return - trades
trades = np.zeros_like(open_prices)
trades


(50, 1452)
[[17.04294995 17.25096871 16.96448663 ... 67.35797313 68.06231506
  68.62432851]
 [12.20377331 12.24209241 12.04319784 ... 27.93657064 28.10079496
  28.43107009]
 [27.76139563 28.32888582 28.12086706 ... 45.28424827 45.45394932
  45.20031238]
 ...
 [12.73111928 12.80593376 12.77856257 ... 40.5016405  40.69141088
  40.9340999 ]
 [ 8.02149948  8.0908393   8.10908629 ... 45.21855798 46.12362377
  46.14186937]
 [20.12126341 20.38584965 20.38584965 ... 57.08475709 57.0592138
  57.54641278]]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:

for stock in range(len(open_prices)):
    #we might have to use open data as the first paramter

    macd, signal_line, _ = ta.MACD(open_prices[stock], fastperiod=12, slowperiod=26, signalperiod=9)

    #if macd crossed above signal line then buy, vice versa sell
    for day in range(1, len(open_prices[0]) - 1):
        if macd[day] > signal_line[day] and macd[day - 1] <= signal_line[day - 1]:
            trades[stock][day + 1] = -1  # Buy signal
        elif macd[day] < signal_line[day] and macd[day - 1] >= signal_line[day - 1]:
            trades[stock][day + 1] = 1  # Sell signal
        else:
            trades[stock][day+1] = 0

In [9]:
from eval_algo import eval_actions

portfolio_value, sharpe_ratio = eval_actions(trades, open_prices, cash=25000, verbose=True)
print(f"\nPortfolio value: {portfolio_value}")
print(f"Sharpe ratio: {sharpe_ratio}")

final portfolio value: 24918.659344478794
cash: 24865.707317163244
positions: [-64.0, -56.0, -44.0, -58.0, -62.0, -55.0, -57.0, 1.0, 0.0, -58.0, -58.0, -61.0, -59.0, -53.0, -60.0, -58.0, -56.0, -60.0, -52.0, -61.0, -54.0, -57.0, 0.0, -51.0, -66.0, 0.0, 1.0, -56.0, 0.0, 0.0, -62.0, 1.0, 0.0, -49.0, 0.0, 0.0, 0.0, -50.0, -52.0, -57.0, -57.0, -58.0, 1.0, 0.0, -60.0, 0.0, 0.0, -60.0, -72.0, 0.0]
short position info: defaultdict(<class 'list'>, {5: [[12.189176550884747, 1.0]], 17: [[14.91531766036664, 1.0]], 38: [], 44: [], 40: [[32.01300997230537, 1.0]], 41: [[25.049851281137915, 1.0]], 1: [], 4: [[48.92275145516362, 1.0]], 10: [[45.71123370217631, 1.0]], 19: [[24.58637186215127, 1.0]], 23: [[60.15577565302906, 1.0]], 27: [[39.96152010016285, 1.0]], 47: [[38.071102873549776, 1.0]], 48: [[44.2970703196363, 1.0]], 11: [[8.84445172619199, 1.0]], 21: [[48.35161659895673, 1.0]], 37: [[7.213145391681668, 1.0]], 12: [[28.038935888679724, 1.0]], 18: [], 39: [[6.256988862193829, 1.0]], 0: [], 2: [[